#Conversation Management & Classification using Groq API


## Task 1: Managing Conversation History with Summarization


In [8]:
# Install latest OpenAI client
!pip install --quiet openai

# Import
from openai import OpenAI

# Ask for API key (runtime only, not saved in notebook)
api_key = input("Paste your GROQ (OpenAI-compatible) API key (will be kept only in this runtime): ")

# Create client pointing to Groq
client = OpenAI(
    api_key=api_key,
    base_url="https://api.groq.com/openai/v1"
)


Paste your GROQ (OpenAI-compatible) API key (will be kept only in this runtime): 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


In [9]:
# Conversation history
conversation_history = []

# Helper to add user/assistant messages
def add_message(role, content):
    conversation_history.append({"role": role, "content": content})


In [10]:
def summarize_conversation(history, model="mixtral-8x7b-32768"):
    """
    Summarize conversation history using Groq API
    """
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a summarizer. Summarize the conversation clearly and concisely."},
            {"role": "user", "content": str(history)}
        ]
    )
    return response.choices[0].message.content


In [11]:
def truncate_history(history, last_n=None, max_chars=None):
    """
    Truncate conversation by last N turns or max characters
    """
    truncated = history

    if last_n:
        truncated = history[-last_n:]
    if max_chars:
        combined = " ".join([msg["content"] for msg in truncated])
        if len(combined) > max_chars:
            # Cut down until under limit
            truncated = []
            current_len = 0
            for msg in reversed(history):
                if current_len + len(msg["content"]) <= max_chars:
                    truncated.insert(0, msg)
                    current_len += len(msg["content"])
                else:
                    break
    return truncated


In [12]:
def simulate_conversation(inputs, k=3, model="mixtral-8x7b-32768"):
    global conversation_history
    run_count = 0

    for user_input in inputs:
        # Add user message
        add_message("user", user_input)

        # Assistant reply (dummy model response for demo)
        response = client.chat.completions.create(
            model=model,
            messages=conversation_history
        )
        assistant_reply = response.choices[0].message.content
        add_message("assistant", assistant_reply)

        run_count += 1

        # Summarize every k-th run
        if run_count % k == 0:
            summary = summarize_conversation(conversation_history, model=model)
            conversation_history = [{"role": "system", "content": "Summary of conversation so far: " + summary}]

        print(f"User: {user_input}")
        print(f"Assistant: {assistant_reply}")
        print("-" * 40)


In [15]:
# Minimal test: single conversation
test_messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello, can you summarize photosynthesis in one line?"}
]

response = client.chat.completions.create(
    model="llama-3.3-70b-versatile",   # Or try another Groq-supported model
    messages=test_messages
)

print(response.choices[0].message.content)


Photosynthesis is the process by which plants, algae, and some bacteria convert sunlight, water, and carbon dioxide into glucose and oxygen.


##Task 2: JSON Schema Classification & Information Extraction

In [16]:
# JSON schema for extracting structured details
schema = {
    "name": "extract_user_info",
    "description": "Extract user details from chat messages",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {"type": "string", "description": "Full name of the person"},
            "email": {"type": "string", "description": "Email address"},
            "phone": {"type": "string", "description": "Phone number"},
            "location": {"type": "string", "description": "User's location (city, country, etc.)"},
            "age": {"type": "integer", "description": "Age of the person"}
        },
        "required": ["name", "email", "phone", "location", "age"]
    }
}


In [17]:
import json

def extract_details_from_chat(chat_text, model="llama-3.3-70b-versatile"):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": chat_text}],
        tools=[
            {
                "type": "function",
                "function": schema
            }
        ],
        tool_choice="auto"  # Let model decide when to call
    )

    # Extract the structured output
    tool_call = response.choices[0].message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)

    return arguments


In [18]:
sample_chats = [
    "Hi, my name is Alice Johnson. I’m 28 years old, based in New York, and you can reach me at alice.j@gmail.com or call me on 123-456-7890.",
    "Hello, this is Raj Singh. I'm 35 from Mumbai, India. My email is raj.singh@yahoo.com and my contact number is +91-9876543210.",
    "Hey there, I’m Maria Lopez, 22, living in Madrid. You can send me an email at maria22@outlook.es or call me at 600123456."
]

for chat in sample_chats:
    details = extract_details_from_chat(chat)
    print("Chat:", chat)
    print("Extracted:", details)
    print("-" * 50)


Chat: Hi, my name is Alice Johnson. I’m 28 years old, based in New York, and you can reach me at alice.j@gmail.com or call me on 123-456-7890.
Extracted: {'age': 28, 'email': 'alice.j@gmail.com', 'location': 'New York', 'name': 'Alice Johnson', 'phone': '123-456-7890'}
--------------------------------------------------
Chat: Hello, this is Raj Singh. I'm 35 from Mumbai, India. My email is raj.singh@yahoo.com and my contact number is +91-9876543210.
Extracted: {'age': 35, 'email': 'raj.singh@yahoo.com', 'location': 'Mumbai, India', 'name': 'Raj Singh', 'phone': '+91-9876543210'}
--------------------------------------------------
Chat: Hey there, I’m Maria Lopez, 22, living in Madrid. You can send me an email at maria22@outlook.es or call me at 600123456.
Extracted: {'age': 22, 'email': 'maria22@outlook.es', 'location': 'Madrid', 'name': 'Maria Lopez', 'phone': '600123456'}
--------------------------------------------------


In [19]:
!pip install --quiet jsonschema

from jsonschema import validate, ValidationError


In [20]:
def validate_extracted_data(data, schema):
    """
    Validate extracted JSON against the defined schema.
    """
    try:
        validate(instance=data, schema=schema["parameters"])
        print("✅ Validation passed:", data)
    except ValidationError as e:
        print("❌ Validation failed:", e.message)
        print("Data was:", data)


In [21]:
sample_chats = [
    "Hi, my name is Alice Johnson. I’m 28 years old, based in New York, and you can reach me at alice.j@gmail.com or call me on 123-456-7890.",
    "Hello, this is Raj Singh. I'm 35 from Mumbai, India. My email is raj.singh@yahoo.com and my contact number is +91-9876543210.",
    "Hey there, I’m Maria Lopez, 22, living in Madrid. You can send me an email at maria22@outlook.es or call me at 600123456."
]

for chat in sample_chats:
    details = extract_details_from_chat(chat)
    print("Chat:", chat)
    validate_extracted_data(details, schema)
    print("-" * 50)


Chat: Hi, my name is Alice Johnson. I’m 28 years old, based in New York, and you can reach me at alice.j@gmail.com or call me on 123-456-7890.
✅ Validation passed: {'age': 28, 'email': 'alice.j@gmail.com', 'location': 'New York', 'name': 'Alice Johnson', 'phone': '123-456-7890'}
--------------------------------------------------
Chat: Hello, this is Raj Singh. I'm 35 from Mumbai, India. My email is raj.singh@yahoo.com and my contact number is +91-9876543210.
✅ Validation passed: {'age': 35, 'email': 'raj.singh@yahoo.com', 'location': 'Mumbai, India', 'name': 'Raj Singh', 'phone': '+91-9876543210'}
--------------------------------------------------
Chat: Hey there, I’m Maria Lopez, 22, living in Madrid. You can send me an email at maria22@outlook.es or call me at 600123456.
✅ Validation passed: {'age': 22, 'email': 'maria22@outlook.es', 'location': 'Madrid', 'name': 'Maria Lopez', 'phone': '600123456'}
--------------------------------------------------
